In [1]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [4]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [5]:
bot = Chatbot(hybrid_retriever,
              system_prompt=academic_system_prompt,
              user_prompt=academic_user_prompt)

In [6]:
query1 = "Cuales son las distintas formas de turismo"
query2= "Hablame de los pasos principales de la organización y el desarrollo de un viaje y los tipos de entidades participantes en ello"

In [7]:
bot.reply(query1)


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Hola! En turismo, se pueden diferenciar varias formas o modalidades, las cuales son:\n\n1. Turismo de playa: Se refiere a viajes al mar o al litoral, con el objetivo de disfrutar de la playa y el clima cálido.\n\n2. Turismo de cultura: Se centra en conocer y apreciar la historia, la arquitectura, el arte y la cultura de un lugar.\n\n3. Turismo de naturaleza: Se enfoca en la exploración y apreciación de los ecosistemas y paisajes naturales, como parques nacionales, reservas naturales o zonas silvestres.\n\n4. Turismo de aventura: Se caracteriza por la participación en actividades de riesgo y desafío, como escalada, rafting, esquí, bungee jumping, etc.\n\n5. Turismo de negocios: Se refiere a viajes realizados para fines de negocio, como conferencias, reuniones, visitas a clientes o proveedores.\n\n6. Turismo de salud: Se enfoca en la recuperación y relajación, con el objetivo de mejorar la salud física y mental, como viajes a spas, centros de yoga, etc.\n\n7. Turismo de eco-turismo: Se 

In [8]:
bot.reply(query2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Hola! La organización y desarrollo de un viaje implica varios pasos principales y la participación de diferentes entidades. Los pasos principales son:\n\n1. Planificación: Se incluye la selección de destino, fecha, duración y actividades a realizar.\n\n2. Investigación: Se busca información sobre el destino, como hoteles, restaurantes, lugares de interés, etc.\n\n3. Reservación: Se reservan vuelos, hoteles, alquiler de vehículos, etc.\n\n4. Documentación: Se obtienen pasaportes, visados, seguros de viaje, etc.\n\n5. Viaje: Se realiza el viaje y se disfruta de las actividades programadas.\n\n6. Retorno: Se regresa al lugar de origen y se realiza la evaluación del viaje.\n\nLas entidades participantes en el desarrollo de un viaje son:\n\n1. Agencias de viajes: Se encargan de la organización y reservación de los servicios necesarios para el viaje.\n\n2. Transporte aéreo: Se refiere a las aerolíneas que ofrecen vuelos para el viaje.\n\n3. Hoteles y alojamientos: Se refieren a los lugares 

In [9]:
bot.history('all')

[{'role': 'system',
  'content': '\nEres un asistente académico experto. \nTu tarea es responder preguntas a los estudiantes sobre distintos aspectos académicos de manera precisa y clara usando el contexto proporcionado. \nSi el contexto es insuficiente, informa al usuario de que no puedes responder su pregunta con la información que posees. Responde en Español.\n--- Contexto ---\n{context}\n'},
 {'role': 'user',
  'content': '\nHola, tengo una pregunta sobre Cuales son las distintas formas de turismo.\n'},
 {'role': 'assistant',
  'content': 'Hola! En turismo, se pueden diferenciar varias formas o modalidades, las cuales son:\n\n1. Turismo de playa: Se refiere a viajes al mar o al litoral, con el objetivo de disfrutar de la playa y el clima cálido.\n\n2. Turismo de cultura: Se centra en conocer y apreciar la historia, la arquitectura, el arte y la cultura de un lugar.\n\n3. Turismo de naturaleza: Se enfoca en la exploración y apreciación de los ecosistemas y paisajes naturales, como p